In [ ]:
# Note: If using billing role following policy needs to be attached
#
# Navigate to IAM -> Policies -> Create Policy
# Select service as Cost Explorer Service
# Tick [X] Read permissions
# Click Review
# Name it, for example MyCostExplorerRead
#     Assign the newly created policy to your user

In [ ]:
import os, json

%reload_ext dotenv
%dotenv
ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID', None)
SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY', None)
SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN', None)
print(ACCESS_KEY)

In [ ]:
import boto3

def get_aws_monthly_bill(start_date, end_date):
    client = boto3.client('ce', region_name='eu-west-1',
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        aws_session_token=SESSION_TOKEN)  # Create a Cost Explorer client

    # Define the request parameters
    request_params = {
        'TimePeriod': {
            'Start': start_date,
            'End': end_date
        },
        'Granularity': 'MONTHLY',
        'Metrics': ['BlendedCost', 'UnblendedCost', 'UsageQuantity']
    }

    # Make the request to retrieve the billing data
    response = client.get_cost_and_usage(**request_params)

    # Extract the billing information for GBP currency
    amount = response['ResultsByTime'][0]['Total']['BlendedCost']['Amount']
    unit = response['ResultsByTime'][0]['Total']['BlendedCost']['Unit']
    usage_quantity = response['ResultsByTime'][0]['Total']['UsageQuantity']['Amount']
    print(json.dumps(response, indent=4))

    return f"The AWS bill for {start_date} to {end_date} is {amount} {unit} (Usage Quantity: {usage_quantity})"


# Usage example: Fetch AWS bill for May 2023
start_date = '2023-04-01'
end_date = '2023-05-30'
bill = get_aws_monthly_bill(start_date, end_date)
print(bill)
